# Explaining Answer Set Solving

This is a (relatively) short guide that shows how the different features/elements of Answer Set Programming work. We will use [clingo](https://potassco.org/clingo/) in Python for this, and throughout this document we will use the syntax that clingo uses for answer set programming.

In [1]:
import clingo

(If you want to learn in more detail about any of the features of answer set programming with clingo that are explained in this guide, please have a look at the official [Potassco guide](https://github.com/potassco/guide/releases/).)

## Printing answer sets

We start by defining a short function that uses the clingo python package to give us the answer sets of a given answer set program (and display the atoms in the answer set sorted alphabetically).

In [2]:
def print_answer_sets(program):
    # Load the answer set program, and call the grounder
    control = clingo.Control()
    control.add("base", [], program)
    control.ground([("base", [])])
    # Define a function that will be called when an answer set is found
    # This function sorts the answer set alphabetically, and prints it
    def on_model(model):
        sorted_model = [str(atom) for atom in model.symbols(shown=True)]
        sorted_model.sort()
        print("Answer set: {{{}}}".format(", ".join(sorted_model)))
    # Ask clingo to find all models (using an upper bound of 0 gives all models)
    control.configuration.solve.models = 0
    # Call the clingo solver, passing on the function on_model for when an answer set is found
    answer = control.solve(on_model=on_model)
    # Print a message when no answer set was found
    if answer.satisfiable == False:
        print("No answer sets")

We can use the function `print_answer_sets()` as follows to print all answer sets of a given answer set program `program`. (We will get to what answer set programs are.)

In [3]:
print_answer_sets("""
    a :- not b.
    b :- not a.
""")

Answer set: {b}
Answer set: {a}


## Answer set semantics

Answer set programming is based on the *answer set semantics* for logic programs. This is easiest explained if we start looking at propositional logic programs. Such a program consists of several rules of the following form:

```
a :- b_1, ..., b_n, not c_1, ..., not c_m.
```

where each of $a$, $b_i$ and $c_i$ are propositional atoms (that start with a lowercase letter). In such a rule, `a` is called the *head* and `b_1, ..., not c_m` is called the *body*. The order of elements within the body of the rule does not matter. This rule is roughly interpreted as: "$a$ is true if $b_1$, ..., $b_n$ are true, and $c_1$, ..., $c_m$ are not true."

You are also free to choose $n=0$, $m=0$, or both. In case both $n=0$ and $m=0$, the rule is simply written as `a.` (and the rule is interpreted as "$a$ is true"). So the following are valid rules in a logic program:

```
a.
a :- b_1, ..., b_n.
a :- not c_1, ..., not c_m.
```

An *interpretation* for a logic program is typically taken to be a set $I$ of propositional atoms. All atoms in the set $I$ are true in this interpretation, and all atoms that are not in $I$ are false in the interpretation.

### Models for positive programs

Answer sets are a particular type of models for logic programs. To explain the particular property that answer sets should have, we first have a look at models for positive programs.

A *positive program* is a logic program that does not contain negations (`not`). In other words, it is a set of rules that are all either of the form `a.` (interpreted as: "$a$ is true") or of the form `a :- b_1, ..., b_n.` (interpreted as "if $b_1$, ..., $b_n$ are true, then $a$ should also be true"). So, a positive program can be seen as a set of facts (`a.`) and logical implications ($(b_1 \wedge \dotsm \wedge b_n) \rightarrow a)$, written as `a :- b_1, ..., b_n`).

A *model* for a positive program is an interpretation $I$ that makes all the logical statements encoded by the rules in the program true. However, some of these models make more sense than others. For example, for the program `a. b :- a.`, the interpretation $I = \{a,b\}$ is a model, but so is $I = \{a,b,c\}$. The first of these two looks reasonable, but the second not so much. For example, to make the rules in the program true, we need to include $a$ and $b$ in the interpretation, but what justification do we have for putting $c$ in?

For positive programs, we are interested in *minimal models*: models $M$ such that no (strict) subset $M' \subseteq M$ also makes the program true. For a positive program $P$, we define the *answer sets* of $P$ to be the set of all minimal models of $P$. It turns out that if a positive program $P$ has a model, then there is always a single unique minimal model of $P$.

So let's try this out:

In [4]:
print_answer_sets("""
    a :- b_1, b_2, b_3.
    b_1.
    b_2 :- c.
    b_3 :- c.
    c.
    d :- e.
    e :- d.
""")

Answer set: {a, b_1, b_2, b_3, c}


Taking the unique minimal model of a positive program nicely matches with our intuition of what we want from models. We include all atoms that are declared to be true in the program (in our example: $a$ and $c$). Then we add other atoms that we must include to make all if-then rules true (for example, adding $b_2$ because we already included $c$ and because `b_2 :- c` is in the program), until we are done.

In our example, we don't include $d$ and $e$, because there is no reason to add them to the model. (We could have added both to the model, and still satisfy all if-then rules.)

### Programs with negation

For logic programs where some rules contain a negation (`not`), things don't turn out to be as easy as simply iterating all if-then rules until we reach a fixpoint. For example, look at the following program:

```
a.
c :- a, not b.
d :- c.
b :- d.
```

If we would use the strategy of applying if-then rules as long as we can, we would first add $a$, then $c$ (due to the rule `c :- a, not b.`), then $d$ (`d :- c.`), and $b$ (`b :- d.`). But then by adding $b$, we made the application of the rule `c :- a, not b.` invalid (because it only works to justify adding $c$ if $b$ is not true).

Nevertheless, we want to select from all the models of a program with negation those models that we are interested in (the answer sets). The idea is to guess an interpretation $I$, use this interpretation to make a version $P^I$ of the program $P$ without negation ($P^I$ is called the *reduct* of $P$ w.r.t. $I$), and then check that $I$ is the unique minimal model of $P^I$—if this is the case, then $I$ is an *answer set* of $P$.

Let's see how this works with a simple example. Take the following program $P$ (lines starting with `%` are comments):
```
% the program P
c.
d :- c, not b.
b :- not d.
```
Take also the interpretation $I_1 = \{c,d\}$. The new program $P^{I_1}$ we get from $P$ by:
1. Removing all rules containing some `not a` in the body such that $a \in I_1$. So in our example, we remove `b :- not d.`, because $d \in I_1$.
1. Removing all remaining statements `not a` from the rest of the program. For all of these statements it holds that $a \not\in I_1$, otherwise we would have removed the entire rule containing the `not a` in step (1). So in our example, we change `d :- c, not b.` into `d :- c.`.

So in our example, the program $P^{I_1}$ becomes:
```
% the reduct P^{I_1}
c.
d :- c.
```

Now, we have that $I_1 = \{c,d\}$ is the unique minimal model of $P^{I_1}$, so $I_1$ is an answer set of $P$.

Let's check with clingo what the answer sets of our example program $P$ are:

In [5]:
print_answer_sets("""
    c.
    d :- c, not b.
    b :- not d.
""")

Answer set: {b, c}
Answer set: {c, d}


Indeed, $\{c,d\}$ is one of the answer sets. Let us also verify why, for example, $I_2 = \{b,c,d\}$ is not an answer set. If we apply the same rules (1) and (2) to $P$ based on $I_2$, we get the program $P^{I_2}$:
```
% the reduct P^{I_2}
c.
```
(We remove both of the rules `d :- c, not b.` and `b :- not d.` because both $b \in I_2$ and $d \in I_2$.)

And since $I_2 = \{b,c,d\}$ is not the unique minimal model of $P^{I_2}$ (which is $\{c\}$), $I_2$ is not an answer set of $P$.

### Answer set semantics for programs with negation

So, to summarize, the answer sets of a propositional logic program $P$, are all interpretations $I$ for which it holds that $I$ is the unique minimal model of $P^I$, where $P^I$ is obtained from $P$ by:
1. Removing all rules containing some `not a` in the body such that $a \in I$.
1. Removing all remaining statements `not a` from the rest of the program.


### Example 0: no answer sets

It turns out that when we allow negation in the logic programs, it is not guaranteed that an answer set exists. Take the following example program $P$:

In [6]:
print_answer_sets("""
    a :- not a.
""")

No answer sets


With only one atom, there are only two relevant interpretations: $I_1 = \emptyset$ and $I_2 = \{a\}$. Both are not answer sets of this program $P$. We get the following two reducts $P^{I_1}$ and $P^{I_2}$:
```
% the reduct P^{I_1}
a.

% the reduct P^{I_2} (empty)
```
Since $I_1$ is not the minimal model of $P^{I_1}$, and $I_2$ is not the minimal model of $P^{I_2}$, neither is an answer set of $P$.

## Some (more) examples

Now that we know what the definition is of answer sets for propositional logic programs, let's look at a few examples to get a bit more feeling for it.

### Example 1: binary choice

Take this example:

In [7]:
print_answer_sets("""
    a :- not b.
    b :- not a.
""")

Answer set: {b}
Answer set: {a}


The rules `a :- not b.` and `b :- not a.` allow us to make a binary choice between $a$ and $b$. If we take $a \in I$, these two rules are replaced by `a.` in $P^I$, and similarly, if we take $b \in I$, the two rules are replaced by `b.` in $P^I$. In other words, we can pick either $a$ or $b$, and $P^I$ will contain a fact justifying our choice.

If we take $I = \{a,b\}$, then $P^I$ becomes the empty program (containing no rules), and $\{a,b\}$ is not the unique minimal model of the empty program (the empty set is), so $\{a,b\}$ is not an answer set of our program.

### Example 2: more binary choice

Now take this example:

In [8]:
print_answer_sets("""
    a :- not b.
    b :- not a.
    
    c :- not d.
    d :- not c.
""")

Answer set: {b, d}
Answer set: {a, d}
Answer set: {b, c}
Answer set: {a, c}


In this case, we took two sets of rules encoding the choice between $a$ and $b$, on the one hand, and $c$ and $d$, on the other hand. The resulting program has four answer sets, corresponding to the four combinations of choices.

### Example 3: overlapping choice

In [9]:
print_answer_sets("""
    a :- not b.
    b :- not a.
    
    a :- not c.
    c :- not a.
""")

Answer set: {b, c}
Answer set: {a}


In this example, we again have two sets of rules that each encode a binary choice. One of these is between $a$ and $b$, and the other between $a$ and $c$. This yields two possible combinations of choices ($a,b$, and $a,c$). The answer sets of this program correspond exactly to these two combinations.

### Example 4: constraints

Suppose now that we want to encode the choices $a/b$ and $c/d$, but that we want to rule out one of the combinations of choices. To do this, we can use a *constraint*, which is a rule with an empty head. For example, the rule `:- a, c.` is a constraint that can be interpreted as "$a$ and $c$ may not both be true". Another way to look at this rule is as the logical implication $(a \wedge c) \rightarrow \bot$, where $\bot$ denotes falsity.

Let's see how we can use this constraint in an example:

In [10]:
print_answer_sets("""
    a :- not b.
    b :- not a.
    
    c :- not d.
    d :- not c.
    
    :- a, c.
""")

Answer set: {b, c}
Answer set: {a, d}
Answer set: {b, d}


### Example 5: another constraint

We can also use negations in constraints. For example, the constraint `:- not a, not c.` can be interpreted as "$a$ and $c$ may not both be false."

In [11]:
print_answer_sets("""
    a :- not b.
    b :- not a.
    
    c :- not d.
    d :- not c.
    
    :- not a, not c.
""")

Answer set: {a, c}
Answer set: {b, c}
Answer set: {a, d}


## Variables

So far, we looked at propositional atoms only. However, in many cases it would be very convenient to use (first-order) variables to range over a certain domain, rather than spelling out everything in the language of propositional logic. Fortunately, this is functionality that we can use.

For example, we can use predicates with constants wherever we used propositional atoms so far. These constants can be numbers, constants (starting with a lowercase letter, e.g., `charlie`), or compound terms built up using function symbols (e.g., `bestfriend(charlie)`).

Note that here the line between predicates (and propositional atoms) and function symbols is blurred a bit: we may use `bestfriend(charlie)` (a) as a unary predicate `bestfriend` applied to the constant `charlie`, or (b) as function symbol `bestfriend` applied to the constant `charlie`.

Note also that terms that are syntactically different from each other are always interpreted as semantically different as well: `bestfriend(charlie)` and `bobbie` are always different (even though one can think of an interpretation where the function symbol `bestfriend` applied to `charlie` is the same as `bobbie`).

Let's phrase one of our previous examples using predicats:

In [12]:
print_answer_sets("""
    choose(a) :- not choose(b).
    choose(b) :- not choose(a).
""")

Answer set: {choose(b)}
Answer set: {choose(a)}


To really use the power of this first-order notation, we can use variables. Variables start with an uppercase letter. Let's start with an example:

In [13]:
print_answer_sets("""
    choice(1).
    choice(2).
    choose(X,a) :- not choose(X,b), choice(X).
    choose(X,b) :- not choose(X,a), choice(X).
""")

Answer set: {choice(1), choice(2), choose(1,b), choose(2,b)}
Answer set: {choice(1), choice(2), choose(1,b), choose(2,a)}
Answer set: {choice(1), choice(2), choose(1,a), choose(2,b)}
Answer set: {choice(1), choice(2), choose(1,a), choose(2,a)}


How does this example work, exactly? Variables are universally quantified, so the rule `a(X) :- b(X).` expresses that for each $c$ for which $b(c)$ holds, also $a(c)$ must hold. What clingo does is spelling out all the different relevant instantiations of rules with variables. This is called *grounding*. (Programs without variables are called *ground programs*.) So under the hood, clingo first changed our previous example into the following:

In [14]:
print_answer_sets("""
    choice(1).
    choice(2).
    choose(1,a) :- not choose(1,b), choice(1).
    choose(1,b) :- not choose(1,a), choice(1).
    choose(2,a) :- not choose(2,b), choice(2).
    choose(2,b) :- not choose(2,a), choice(2).
""")

Answer set: {choice(1), choice(2), choose(1,b), choose(2,b)}
Answer set: {choice(1), choice(2), choose(1,a), choose(2,b)}
Answer set: {choice(1), choice(2), choose(1,b), choose(2,a)}
Answer set: {choice(1), choice(2), choose(1,a), choose(2,a)}


### Safe rules

In order to make sure that the grounding process works, you can only use rules that are *safe*. What this means is that every variable that appears in the rule must appear in some positive (that is, non-negated) element of the body. So for example, the following rules are *unsafe* (because the variable `Y` does not appear positively in the body):

```
a(Y) :- not b(Y).
a(X,Y) :- c(X).
```

If you ask clingo to find answer sets for a program that contains unsafe rules, it will throw an error message.

## Abbreviations and other additional features

We have seen all the basic features of answer set programming. However, clingo and the language of answer set programming make our life easier by providing some further features. Let's look at some of them by means of some further examples.

### Example 6: enumerating numbers

Suppose that we want to declare `choice(i)` for all integers between 1 and 10. Rather than spelling out ten facts, we can simply write `choice(1..10).` So we can write our previous choice example as follows:

In [15]:
print_answer_sets("""
    choice(1..2).
    choose(X,a) :- not choose(X,b), choice(X).
    choose(X,b) :- not choose(X,a), choice(X).
""")

Answer set: {choice(1), choice(2), choose(1,b), choose(2,b)}
Answer set: {choice(1), choice(2), choose(1,b), choose(2,a)}
Answer set: {choice(1), choice(2), choose(1,a), choose(2,b)}
Answer set: {choice(1), choice(2), choose(1,a), choose(2,a)}


### Example 7: showing only some predicates in the answer sets

In our choice example, we might be interested in only part of the answer set. Namely, in the binary predicate `choose`. We know that every answer set will contain `choice(1)`, etc. We can declare a statement that says we want to show the binary predicate `choose`: `#show choose/2.`. If you issue one or more show statements, then all predicates for which no show statement is issued are automatically hidden from answer sets:

In [16]:
print_answer_sets("""
    choice(1..2).
    choose(X,a) :- not choose(X,b), choice(X).
    choose(X,b) :- not choose(X,a), choice(X).
    #show choose/2.
""")

Answer set: {choose(1,b), choose(2,b)}
Answer set: {choose(1,b), choose(2,a)}
Answer set: {choose(1,a), choose(2,b)}
Answer set: {choose(1,a), choose(2,a)}


### Example 8: declaring constants

Suppose that we want to use a number that we will use more often in a program, and that we want to be able to change it easily in a single place. Then we can declare this number as a constant, as follows:

In [17]:
print_answer_sets("""
    #const k=2.
    choice(1..k).
    choose(X,a) :- not choose(X,b), choice(X).
    choose(X,b) :- not choose(X,a), choice(X).
    #show choose/2.
""")

Answer set: {choose(1,b), choose(2,b)}
Answer set: {choose(1,b), choose(2,a)}
Answer set: {choose(1,a), choose(2,b)}
Answer set: {choose(1,a), choose(2,a)}


In [18]:
print_answer_sets("""
    #const k=3.
    choice(1..k).
    choose(X,a) :- not choose(X,b), choice(X).
    choose(X,b) :- not choose(X,a), choice(X).
    #show choose/2.
""")

Answer set: {choose(1,b), choose(2,b), choose(3,b)}
Answer set: {choose(1,b), choose(2,a), choose(3,b)}
Answer set: {choose(1,b), choose(2,b), choose(3,a)}
Answer set: {choose(1,b), choose(2,a), choose(3,a)}
Answer set: {choose(1,a), choose(2,b), choose(3,b)}
Answer set: {choose(1,a), choose(2,b), choose(3,a)}
Answer set: {choose(1,a), choose(2,a), choose(3,b)}
Answer set: {choose(1,a), choose(2,a), choose(3,a)}


### Example 9: abbreviating facts

Instead of using `choice(1..3).`, we could also use the statement `choice(1;2;3).`:

In [19]:
print_answer_sets("""
    choice(1;2;3).
""")

Answer set: {choice(1), choice(2), choice(3)}


### Example 10: choice rules

Suppose that we want a program that encodes a choice between all subsets of a given set. For example, let $A = \{a_1,a_2\}$ and suppose that we want to write a program whose answer sets (restricted to some predicate `choose/1`) correspond exactly to the four subsets of $A$. We can do this as follows, using the technique that we saw in the examples about binary choice:

In [20]:
print_answer_sets("""
    element(a;b).
    choose(X) :- not unchoose(X), element(X).
    unchoose(X) :- not choose(X), element(X).
    #show choose/1.
""")

Answer set: {}
Answer set: {choose(b)}
Answer set: {choose(a)}
Answer set: {choose(a), choose(b)}


However, we can also encode this, more conveniently, using so-called choice rules. For example, the rule `{ choose(a;b) }.` represents exactly what the above example did:

In [21]:
print_answer_sets("""
    { choose(a;b) }.
""")

Answer set: {}
Answer set: {choose(b)}
Answer set: {choose(a)}
Answer set: {choose(a), choose(b)}


Or, equivalently, written as:

In [22]:
print_answer_sets("""
    { choose(a); choose(b) }.
""")

Answer set: {}
Answer set: {choose(b)}
Answer set: {choose(a)}
Answer set: {choose(a), choose(b)}


You can use these choice rules in the head of a rule, also with a non-empty body of the rule (e.g., `{ choose(a;b) } :- make_choice.`).

### Example 11: cardinality rules

A construct that is similar to choice rules is that of cardinality rules. These are rules that encode a choice between subsets of a given set that satisfies some cardinality conditions. For example, we can modify the example above so that only subsets of size at least 1 and at most 2 are given:

In [23]:
print_answer_sets("""
    1 { choose(a); choose(b) } 2.
""")

Answer set: {choose(a)}
Answer set: {choose(b)}
Answer set: {choose(a), choose(b)}


We can also use these cardinality expressions in the body of a rule:

In [24]:
print_answer_sets("""
    { choose(a); choose(b) }.
    exactly_one :- 1 { choose(a;b) } 1.
""")

Answer set: {}
Answer set: {choose(b), exactly_one}
Answer set: {choose(a), exactly_one}
Answer set: {choose(a), choose(b)}


### Example 12: conditional literals

Another convenient feature is the use of conditional literals. This we can use to express a set of statements for which another property is true. This works as in the following example, where we declare several items using `item/1`, and then encode a choice of exactly two of these items using a cardinality rule where we express the set using conditional literals:

In [25]:
print_answer_sets("""
    item(a;b;c).
    2 { choose(X) : item(X) } 2.
    #show choose/1.
""")

Answer set: {choose(b), choose(c)}
Answer set: {choose(a), choose(c)}
Answer set: {choose(a), choose(b)}


### Example 13: arithmetic

You can use integers as constants. Moreover, you can use arithmetic operations (e.g., comparing integers, addition, multiplication, etc).

In [26]:
print_answer_sets("""
    number(1..10).
    four(4).
    at_most_five(N) :- number(N), four(F), N <= F+1.
    #show at_most_five/1.
""")

Answer set: {at_most_five(1), at_most_five(2), at_most_five(3), at_most_five(4), at_most_five(5)}


However, these arithmetic operations can only be applied to variables whose value can be determined by means of other predicates. Therefore, if arithmetic operations involve variables that do not appear in a positively occurring atom in the body of the rule, clingo will throw an error message. (Such rules are not *safe*, as we explained before.)

In the following example, the rule for `double(X,Y)` contains arithmetic operations on `X` and `Y`, and these variables do not occur in a positive atom in the body, and so the rule is not safe and clingo will throw an error message.

In [27]:
try:
    print_answer_sets("""
        number(1..10).
        double(X,Y) :- not letter(X), not letter(Y), Y = 2*X.
        #show double/2.
    """)
except RuntimeError as e:
    print("RuntimeError: \"{}\"".format(str(e)))

RuntimeError: "grounding stopped because of errors"


We can make the rule for `double(X,Y)` safe as follows, for example.

In [28]:
print_answer_sets("""
    number(1..10).
    double(X,Y) :- number(X), number(Y), Y = 2*X.
    #show double/2.
""")

Answer set: {double(1,2), double(2,4), double(3,6), double(4,8), double(5,10)}


### Example 14: aggregates

Another useful feature that clingo offers is the use of *aggregates*: `#sum`, `#count`, `#max`, `#min`, `#even`, and `#odd`. These aggregates operate on a (multi)set of atoms.

They can be used as follows, for example:

In [29]:
print_answer_sets("""
    item(1..4).
    cost(1,2).
    cost(2,2).
    cost(3,3).
    cost(4,0).
    2 { choose(I) : item(I) } 2.
    total(D) :- D = #sum { C,item(I) : item(I), choose(I), cost(I,C) }.
    #show total/1.
    #show choose/1.
""")

Answer set: {choose(1), choose(4), total(2)}
Answer set: {choose(2), choose(4), total(2)}
Answer set: {choose(1), choose(2), total(4)}
Answer set: {choose(3), choose(4), total(3)}
Answer set: {choose(2), choose(3), total(5)}
Answer set: {choose(1), choose(3), total(5)}


In the above example, we use `C,item(I)` in the `#sum` aggregate to make sure that whenever there are more items with the same cost, all of their costs are counted towards the total. If we were to replace `C,item(I)` by `C`, several items with the same cost `C` would (all together) only contribute `C` to the sum.

Here is another example, now using the `#count` aggregate.

In [30]:
print_answer_sets("""
    item(1..4).
    total(D) :- D = #count { item(I) : item(I) }.
""")

Answer set: {item(1), item(2), item(3), item(4), total(4)}


Finally, an example using `#max`:

In [31]:
print_answer_sets("""
    item(1..4).
    2 { choose(I) : item(I) } 2.
    highest(D) :- D = #max { I : item(I), choose(I) }.
    #show highest/1.
    #show choose/1.
""")

Answer set: {choose(1), choose(2), highest(2)}
Answer set: {choose(1), choose(3), highest(3)}
Answer set: {choose(2), choose(3), highest(3)}
Answer set: {choose(1), choose(4), highest(4)}
Answer set: {choose(2), choose(4), highest(4)}
Answer set: {choose(3), choose(4), highest(4)}


## Optimization

We can also use optimization statements, to select from all answer sets of a given answer set program those answer sets that minimize or maximize a particular property. To illustrate this, we will define a short function that will give us all optimized answer sets for a given answer set program.

In [32]:
def print_optimal_answer_sets(program):
    # Load the answer set program, and call the grounder
    control = clingo.Control()
    control.add("base", [], program)
    control.ground([("base", [])])
    # Define a function that will be called when an answer set is found
    # This function sorts the answer set alphabetically, and prints it
    def on_model(model):
        if model.optimality_proven == True:
            sorted_model = [str(atom) for atom in model.symbols(shown=True)]
            sorted_model.sort()
            print("Optimal answer set: {{{}}}".format(", ".join(sorted_model)))
    # Ask clingo to find all optimal models (using an upper bound of 0 gives all models)
    control.configuration.solve.opt_mode = "optN"
    control.configuration.solve.models = 0
    # Call the clingo solver, passing on the function on_model for when an answer set is found
    answer = control.solve(on_model=on_model)
    # Print a message when no answer set was found
    if answer.satisfiable == False:
        print("No answer sets")

With this function in place, we will illustrate how optimization statements work.

Consider the following example, where we have facts declaring four items and a score for each of these items. We also have a cardinality rule that states that we should select between 2 and 3 of these items. This gives us 10 answer sets in total:

In [33]:
print_answer_sets("""
    item(1..4).
    score(1,5).
    score(2,4).
    score(3,4).
    score(4,2).
    2 { select(I) : item(I) } 3.
    #show select/1.
""")

Answer set: {select(3), select(4)}
Answer set: {select(2), select(3)}
Answer set: {select(2), select(4)}
Answer set: {select(2), select(3), select(4)}
Answer set: {select(1), select(2)}
Answer set: {select(1), select(4)}
Answer set: {select(1), select(2), select(4)}
Answer set: {select(1), select(3)}
Answer set: {select(1), select(3), select(4)}
Answer set: {select(1), select(2), select(3)}


Now let's add an optimization statement, that states that we should maximize the total score for the items that we select. This shows that only 2 of the 10 answer sets maximize this total score.

In [34]:
print_optimal_answer_sets("""
    item(1..4).
    score(1,5).
    score(2,4).
    score(3,4).
    score(4,2).
    2 { select(I) : item(I) } 3.
    #maximize { S,I : select(I), score(I,S) }.
    #show select/1.
""")

Optimal answer set: {select(1), select(2), select(3)}


How this works is as follows. The statement `{ S,I : select(I), score(I,S) }` refers to the set of all pairs `S,I` for which `select(I), score(I,S)` holds in the answer set. Then the `#maximize` statements indicates that only those answer sets should be taken for which the sum of all `S` in such pairs `S,I` is maximal. You may also use `#maximize` statements with tuples of different arity (e.g., triples, or 1-tuples). In this case, however, we need to include `I` in the tuples, because otherwise the score for item 2 and item 3 would only be counted once in the total score.

This works similarly using `#minimize` instead of `#maximize`.

In [35]:
print_optimal_answer_sets("""
    item(1..4).
    score(1,5).
    score(2,4).
    score(3,4).
    score(4,2).
    2 { select(I) : item(I) } 3.
    #minimize { S,I : select(I), score(I,S) }.
    #show select/1.
""")

Optimal answer set: {select(2), select(4)}
Optimal answer set: {select(3), select(4)}


To see why we need the tuple `S,I` in this `#minimize` statement, consider the following example, where we replace `S,I` by just `S` in the `#minimize` statement:

In [36]:
print_optimal_answer_sets("""
    item(1..4).
    score(1,5).
    score(2,4).
    score(3,4).
    score(4,2).
    2 { select(I) : item(I) } 3.
    #minimize { S : select(I), score(I,S) }.
    #show select/1.
""")

Optimal answer set: {select(2), select(3)}


Now selecting items 2 and 3 yields the mimimal total score, because both add score 4 to the set `{ S : select(I), score(I,S) }`, and now the score 4 gets only counted once, which is not what we had in mind.

## Disjunction

There is an extension of answer set programming that allows us to use disjunction in the head of rules, and clingo supports this. The following example illustrates how we can do this (with the operator `;` in the head of a rule):

In [37]:
print_answer_sets("""
    a.
    b ; c :- a.
""")

Answer set: {a, b}
Answer set: {a, c}


To make sure that we have a good foundation for this, we make sure that our definition of *answer sets* also works for programs with rules that have disjunction in the head.

The idea is the same. We guess an interpretation $I$, use this interpretation to make a version $P^I$ of the program $P$ without negation (the *reduct* of $P$ w.r.t. $I$), and then check that $I$ is a minimal model of $P^I$—if this is the case, then $I$ is an *answer set* of $P$.

The reduct $P^{I}$ we get from $P$ by:
1. Removing all rules containing some `not a` in the body such that $a \in I$.
1. Removing all remaining statements `not a` from the rest of the program.
So this is also exactly the same as for the case without disjunction in the head of rules.

The only thing that we need to update is what it means for $I$ to be a (minimal) model of $P^I$. We consider rules `a ; b :- c_1, ..., c_n` as the logical implication $(c_1 \wedge \dotsm \wedge c_n) \rightarrow (a \vee b)$. That is, we interpret disjunction in the head as logical disjunction in the consequent of the implication. Using this, we can use a similar definition of what an answer set is: an interpretation $I$ that is a minimal (w.r.t. set inclusion) model of the reduct $P^I$ of the program $P$ w.r.t. $I$.

To take an example, consider the program that we used as example above:
```
% the program P
a.
b ; c :- a.
```
Consider the interpretation $I_1 = \{a,b\}$. The reduct $P^{I_1}$ is:
```
% the reduct P^{I_1}
a.
b ; c :- a.
```
And $I_1$ is in fact a minimal model of $P^{I_1}$. If we remove $a$ from $I_1$, we get the interpretation $\{b\}$, which does not satisfy `a.`. This is also the case if we remove $a$ and $b$ from $I_1$. If we instead only remove $b$ from $I_1$, we get the interpretation $\{a\}$, which does not satisfy `b ; c :- a.`. Thus, $I_1$ is a minimal model of $P^{I_1}$.

### Using default negation to represent disjunctive choices

As we have seen above, we can add disjunction to the head of rules. However, in many cases this is not needed to express disjunctive choices. For example, as we have already seen earlier, we can encode a choice between two atoms (say, `a` and `b`) using several rules as follows:

In [38]:
print_answer_sets("""
    a :- not b.
    b :- not a.
""")

Answer set: {b}
Answer set: {a}
